In [1]:
import re
from collections import Counter
import itertools as it

In [2]:
def words(text): return re.findall(r"\w+", text.lower())

In [3]:
# read corpus
text = words(open('ind_news_2012_300K-sentences.txt',encoding='utf8').read())

In [4]:
# counter words for bigram
WORDS_2 = Counter(zip(text, it.islice(text, 1, None)))

In [5]:
# counter words for unigram
WORDS = Counter(text)

In [6]:
# simple edit : to a word is a deletion, a transposition, a replacement or an insertion
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

In [7]:
#words that are known—that is, in the dictionary
def known(words): return set(w for w in words if w in WORDS)

In [8]:
# 2 x simple edit : to a word is a deletion, a transposition, a replacement or an insertion
def edits2(word): return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [9]:
# get the word candidate list from error word 
def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

In [10]:
# probability for unigram
def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

In [11]:
# probability for bigram, given two words
def P2(words, N=sum(WORDS_2.values())):
    V = len(WORDS.values())
    "Probability of two `words --> for bigram`."
    #P(B|A) =  C(A|B) + 1/P(B) + V #with smoothing
    return ((WORDS_2[words[0], words[1]]/N)+1)/(P(words[0]) + V)

In [12]:
# find a correction word for unigram
def correction(word): 
    "Most probable spelling correction for word."
    # a corection word will be chosen from the words candidate which has max probability
    return max(candidates(word), key=P)

In [13]:
# find a correction word for bigram
def correction_2(words):
    #check candidate list
    word_candidates = list(candidates(words[1]))
    #temp -> index of candidate list which has bigger probability
    max_index = 0
    #temp -> current max probability
    max_prob = 0
    #iterating candidate list (find max probability)
    for index, word in enumerate(word_candidates):
        if P2([words[0], word])*P2([word, words[2]]) > max_prob :
            max_index = index
            #P(b, a)*p(a, c)
            max_prob = P2([words[0], word])*P2([word, words[2]])
    #high probability candidate will be chosen
    return word_candidates[max_index]

In [14]:
# spelling test for unigram 
def spelltest(tests, verbose=False):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.perf_counter()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correction(wrong)
        good += (w == right)
        if w != right:
            unknown += (right not in WORDS)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'
                      .format(wrong, w, WORDS[w], right, WORDS[right]))
    dt = time.perf_counter() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))

In [15]:
# test set function for given test data
def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

In [16]:
# unigram testing
spelltest(Testset(open('testset.txt')))

17% of 225 correct (53% unknown) at 35 words per second 


In [17]:
# spelling test for bigram
def spelltest_2(tests, verbose=False):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.perf_counter()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correction_2(wrong)
        #delete white space
        right = right.strip()
        good += (w == right)
        if w != right:
            unknown += (right not in WORDS)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'
                      .format(wrong, w, WORDS[w], right, WORDS[right]))
    dt = time.perf_counter() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))

In [18]:
# test set function for bigram by given test data
def Testset2(lines):
    #array of test set
    array_test2 = []
    for line in lines:
        #the right words
        right = (line.split(':')[0].lower())
        #a sentence that need to be corected in the midle of word
        wrong = (line.split(':')[1].rstrip().split())
        #to lowercase
        wrong = [word.lower() for word in wrong]
        #add it to tuple
        y = (right, wrong)
        array_test2.append(y)
    return array_test2

In [19]:
# bigram testing using my own dataset
spelltest_2(Testset2(open('testset2.txt')))

25% of 16 correct (0% unknown) at 2367 words per second 


In [20]:
#bigram perform more accurately than unigram
#because in bigram not just cosidering one word itself
#but In bigram we consider past one word and the next word
#so from the bigram language model, we can get the context of the word that needs a correction, and corected it more accurately 